In [ ]:
#Autor: Octavio Osorio Trujillo
#Correo: octavioosoriotrujillo@gmail.com
#Fecha de creación: 17/08/2025
#Última modificación: 27/08/2025
#Descripción: Script para procesar datos de mareógrafico y generar gráficos interactivos
#Versión: 0

In [1]:
#PASO 1: EJECUTE PRIMERO ESTA CELDA

#Este repositorio nos sirve para ejecutar los scripts: programas independientes que ayudan en el analisis del archivo
#seleccionado y ademas se realiza la importacion de modulos: donde se reautiliza codigo, su ejecucion  es necesaria
!git clone https://github.com/Octavio341/MareograficoGoogleColab.git

Cloning into 'MareograficoGoogleColab'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 333 (delta 40), reused 65 (delta 39), pack-reused 259 (from 1)
Receiving objects: 100% (333/333), 2.77 MiB | 8.62 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [ ]:
#############################################################################
#[0]=================  Cuadros y Grafica Dividida ==========================
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
import numpy as np
#############################################################################
#[0]=================  Ventana y cuadros Y CARGAR ARCHIVOS ==========================
import ipywidgets as widgets
from IPython.display import display,clear_output,HTML
#############################################################################
#[0]=================  CARGAR ARCHIVOS y Grafica Dividida ==========================
import os
from google.colab import files, drive
import hashlib
#############################################################################
#[0]==================== Grafica Dividida =================================
import pandas as pd
from bokeh.models import Button as BokehButton,CustomJS,Scatter,LegendItem, HoverTool,Label,CrosshairTool,CheckboxGroup,ColumnDataSource, TableColumn, DataTable, DateFormatter,TabPanel, Tabs, Div,Select,TextInput,LabelSet,Slider, DatetimeTickFormatter,Checkbox,RadioButtonGroup
from bokeh.models.widgets import Button
from bokeh.palettes import Category20
from bokeh.layouts import column, row,gridplot#para la posiciones
from bokeh.embed import components
from bokeh.resources import CDN
import datetime
import ipywidgets as widgets
#para ejecutar scripts
import subprocess
#############################################################################
#==================== SINCRONIZACION DE ARCHIVOS =================================
import os
import sys
import importlib
import ipywidgets as widgets
#
import time

script_dest = '/content/MareograficoGoogleColab/ET1_ proyecto 1/GENERAL/SCRIPTS -- de practica'
#SI AQUI TE SALE UN ERROR ES POR QUE NO HAS EJECUTADO LA PRIMERA CELDA
#se tiene que realizar una llamada al repositorio para pasar de este punto
print(os.listdir(script_dest))
sys.path.append(script_dest)

import sincronizado
import importlib
importlib.reload(sincronizado)  # fuerza recarga del módulo | para evitar el caché:

from sincronizado import super_resumen
#############################################################################
#==================== Grafica Dividida =================================

output_notebook()

Controles_zoom = widgets.Output()
estado = {}

# Lista global de archivos procesados
archivos_procesados = []
archivos_procesado_2 = []
archivos_procesados.clear()
archivos_procesado_2.clear()   # ✅ borra todos los elementos de la lista

#############################################################################
#=================  Cuadros  ==========================
# diccionario de estilos CSS en Python, para pesonalizar la apariencia
# > de un cuadro(contenedor)
style_box = {
    'border': '5px solid #2D6FA4',
    'padding': '10px',
    'margin': '1.5px',
    'border-radius': '8px',
    'background-color': '#10283B',  #
    'width': '570px',
    'height': '280px',
    'overflow': 'auto',
}

#todo el contenido del cuadrito
def make_box(title, content_widget):
    label = widgets.HTML(value=f"<b>{title}</b>")
    # VBox para apilar el título y el contenido
    box = widgets.VBox([label, content_widget])
    box.layout = widgets.Layout(
        border=style_box['border'],
        padding=style_box['padding'],
        margin=style_box['margin'],
        border_radius=style_box['border-radius'],
        background_color=style_box['background-color'],
        width=style_box['width'],
        height=style_box['height'],
        overflow=style_box['overflow'],
    )
    return box
#############################################################################
#==================== Grafica Dividida, interacciones de la grafica =================================
#mostramos la grafica
ejecucion_script = widgets.Output()
def actualizar_vista():
  clear_output(wait=True)
  nombres_archivos = []
  herramientas=[
        "pan",
        "wheel_zoom",
        "box_zoom",
        "reset",
        "save",
        "lasso_select",
        "box_select",
        "tap",
        "zoom_in",
        "zoom_out",
        "help"
      ]

  #crea una figura de Bokeh con configuración personalizada
  p = figure(
    x_axis_label="Fecha",
    y_axis_label="Nivel del mar (mm)",
    x_axis_type="datetime",
    width=1300,
    height=600,
    tools=','.join(herramientas),
    toolbar_location="above",
    toolbar_sticky=False,
    margin=(5, 5, 5, 5)
  )


  #Añadir CrosshairTool (Muestra lineas guia horizontal y verticales para facilitar lectura)
  crosshair = CrosshairTool(dimensions= 'both')#lineas vertical y horizontal
  p.add_tools(crosshair)
  #############################################################################
  #==================== PERSONALIZAMOS LA TABLA =================================

  # Ejemplo: Cambiar color de texto en una columna con HTMLTemplateFormatter
  css_personalizado = """
  .bk-data-table {
      background-color: #ffffff !important;
      color: #000000 !important;
      font-size: 16px !important;
      font-weight: normal !important;
      border: 9px solid #ddd !important;
      border-radius: 8px !important;
      font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif !important;
  }
  .bk-header {
      background-color: #f4d03f !important;
      color: #f9f9f9 !important;
      font-weight: bold !important;
      font-size: 18px !important;
  }
  .bk-data-table .slick-row:nth-child(even) {
      background-color: #595959 !important;
  }
  .bk-data-table .slick-row:nth-child(odd) {
      background-color: #595959 !important;
  }
  /* Selección destacada */
  .bk-data-table .slick-row.selected {
      background-color: #fffb91 !important;
      color: #000000 !important;
      font-weight: 900 !important;
      box-shadow: 0 0 10px 3px rgba(255, 255, 145, 0.6) !important;
      border: 2px solid #e1d700 !important;
      outline: none !important;
      filter: none !important;
      opacity: 1 !important;
      transition: background-color 0.3s ease, box-shadow 0.3s ease, border 0.3s ease;
  }
  .bk-data-table .slick-row:hover {
      background-color: rgba(255, 235, 59, 0.3) !important;
      color: #000000 !important;
      font-weight: 700 !important;
      box-shadow: inset 0 0 10px rgba(255, 235, 59, 0.4) !important;
      outline: 1px solid rgba(255, 235, 59, 0.7) !important;
      filter: none !important;
      opacity: 1 !important;
      cursor: pointer;
      transition: background-color 0.3s ease, box-shadow 0.3s ease, outline 0.3s ease;
  }
  .bk-data-table .slick-cell {
      border-right: 1px solid rgba(200, 200, 200, 0.5) !important;
  }
  .bk-data-table .slick-cell:last-child {
      border-right: none !important;
  }

  """
  #############################################################################
  #==================== EMPEZAMOS A CREAR LA GRAFICA Y TODOS LOS FILTROS =================================
  #Uso de la tabla
  colores = Category20[20]
  source2 = []
  lineas = [] #para guardar las lineas

  pico_e = []
  valles_e = []
  nombres_archivos = []
  puntos = []
  all_labels = []
  sources_E = []
  nombre=[]

  linea33 = []
  linea44 = []


  for i, (df, archivo) in enumerate(zip(dfs, archivos_cargados)):
    labels_originales = [str(v) for v in df["Registro"]]
    source2 = ColumnDataSource(data=dict(
        x=pd.to_datetime(df['Date']),
        y=df['Registro'],
        labels=[""] * len(df["Registro"])

    ))
    # Inicializar la bolita en el primer punto

    # ColumnDataSource de la bolita
    source_ball = ColumnDataSource(data=dict(
        x=[df['Date'].iloc[0]],
        y=[df['Registro'].iloc[0]]
    ))

    source_rango = ColumnDataSource(data=dict(
        x=[df['Date'].iloc[0]],
        y=[df['Registro'].iloc[0]]
    ))

    source_stuck = ColumnDataSource(data=dict(
      x_stuck=df['Date_stuck'],
      y_stuck=df['Registro_stuck']
    ))


    # Calcular incremento
    df = pd.DataFrame(source2.data)  # conviertes los datos a DataFrame
    # Calcular incremento
    df["incremento"] = df["y"].diff()    # Actualizar el ColumnDataSource
    source2.data = df.to_dict(orient="list")

    source3 = source2
    #############################################################################
    #============================== LINEAS ORIGINALES =========================
    colores = ["black","green","orange","purple","blue","gray","white","brown","pink","cyan"]
    start_index = 0  # negro es el primer color

    nombre = os.path.basename(archivo)
    nombres_archivos.append(nombre)#guardamos el nombre del archivo

    linea2 = p.scatter('x', 'y', source=source2, legend_label='Registro original', line_width=5, color=colores[(i + start_index) % len(colores)]) #las graficas con puntos
    linea1 = p.line(source=source2, line_width=2,color="blue") #guardamos las lineas de nuestra grafica
    #################################################################################
    #============================== PROMEDIO, SIGMA Y MARGEN =========================
    # Tomamos los datos de source2
    x_vals_all = np.array(source2.data['x'])
    y_vals_all = np.array(source2.data['y'])

    # Filtrar solo los datos válidos (no nulos, no NaN)
    mask_valid = (~pd.isnull(x_vals_all)) & (~pd.isnull(y_vals_all))

    x_vals = x_vals_all[mask_valid]
    y_vals = y_vals_all[mask_valid]

    # Calcular promedio y sigma
    prom_orig = np.mean(y_vals)
    sigma_orig = np.std(y_vals)

    # Radio buttons para elegir rango
    radio_sigma = RadioButtonGroup(labels=["±1σ", "±2σ", "±3σ"], active=0)

    # Filtrar usando source
    min_orig = prom_orig - sigma_orig
    max_orig = prom_orig + sigma_orig
    mask = (y_vals >= min_orig) & (y_vals <= max_orig)

    # Datos filtrados
    x_f_orig = x_vals[mask]
    y_f_orig = y_vals[mask]

    source_in_range = ColumnDataSource(data=dict(x=x_f_orig, y=y_f_orig))
    ##################################################################################
    #============================== DENTRO DEL RANGO =================================
    renderer_in_range = p.scatter('x','y',source= source_in_range,size=10, color="red", alpha=0.8, legend_label="Dentro del rango")

    #Usar los extremos de x_vals(que viene de source2)
    if len(x_vals) > 0:
        x_min, x_max = x_vals[0], x_vals[-1]
        # Tomar el año mínimo de tus datos
        # Convierte el mínimo a datetime de Python
        x_min_dt = pd.to_datetime(x_vals.min()).to_pydatetime()
        anio = x_min_dt.year
        fecha_fija = datetime.datetime(anio,1,1,12,0)
    else:
        print("⚠️ x_vals está vacío, se usarán valores por defecto.")
        print("")
        # Definir un rango ficticio para que la gráfica no falle
        fecha_fija = pd.to_datetime("2000-01-01")

        # --- Siempre aseguramos valores por defecto ---
        x_min = pd.to_datetime("2000-01-01")
        x_max = pd.to_datetime("2000-12-31")
        #esto es para evitar advertencias
        prom_orig, min_orig, max_orig = 0, 0, 0
    if len(y_vals) > 0 and not np.all(np.isnan(y_vals)):
        prom_orig = np.nanmean(y_vals)
        min_orig  = np.nanmin(y_vals)
        max_orig  = np.nanmax(y_vals)
    else:
      prom_orig, min_orig, max_orig = 0, 0, 0


   #Lineas etiquetadas
    linea_prom = p.line([x_min, x_max], [prom_orig, prom_orig],
                    line_color="red", line_width=2, line_dash="dashed", legend_label="Promedio")
    linea_min = p.line([x_min, x_max], [min_orig, min_orig],
                      line_color="black", line_width=2, line_dash="dotted", legend_label="Mínimo")
    linea_max = p.line([x_min, x_max], [max_orig, max_orig],
                      line_color="black", line_width=2, line_dash="dotted", legend_label="Máximo")

    #print("Anio",anio)
    #---------------------=Ahora el label se ubicará sobre la fecha 1958-01-01 12:00 en el eje X.
    label_prom = Label(x=fecha_fija, y=prom_orig, text=f"Promedio: {prom_orig:.2f}", text_color="red", text_font_size="15pt")
    label_min = Label(x=fecha_fija, y=min_orig, text=f"Mín: {min_orig:.2f}", text_color="black", text_font_size="15pt")
    label_max = Label(x=fecha_fija, y=max_orig, text=f"Máx: {max_orig:.2f}", text_color="black", text_font_size="15pt")
    p.add_layout(label_prom)
    p.add_layout(label_min)
    p.add_layout(label_max)

    ##################################################################################
    #============================== CASILLAS =================================
    # Puntos filtrados
    renderer_in_range.visible = False

    linea_prom.visible = False
    linea_min.visible = False
    linea_max.visible = False


    # Labels
    label_prom.visible = False
    label_min.visible = False
    label_max.visible = False

    chk_todo = Checkbox(label="Mostrar rango y promedio", active=False)

    chk_todo.js_on_change("active", CustomJS(args=dict(
    r=renderer_in_range,
    l_prom=linea_prom, l_min=linea_min, l_max=linea_max,
    lab_prom=label_prom, lab_min=label_min, lab_max=label_max
    ), code="""
        const estado = cb_obj.active;
        r.visible = estado;
        l_prom.visible = estado;
        l_min.visible = estado;
        l_max.visible = estado;
        lab_prom.visible = estado;
        lab_min.visible = estado;
        lab_max.visible = estado;
    """))
    ##################################################################################
    #============================== control de picos y valles =======================

    from scipy.signal import find_peaks
    indices_picos, _ = find_peaks(source3.data["y"], distance=6)
    # Mínimos locales (bajamares)
    indices_valles, _ = find_peaks(-np.array(source3.data['y']), distance=6)

    df["pico"] = np.nan
    # --- Asignar valores a las columnas pico y valle dentro de source2 ---
    # Crear columnas 'pico' y 'valle' en source2 (inicialmente NaN)
    source3.data['pico'] = [np.nan] * len(source3.data['y'])
    source3.data['valle'] = [np.nan] * len(source3.data['y'])
    for ii in indices_picos:
        source3.data['pico'][ii] = source3.data['y'][ii]

    for ii in indices_valles:
        source3.data['valle'][ii] = source3.data['y'][ii]
    ##################################################################################
    #============================== CREAMOS LAS COLUMNAS DE LA TABLA =================
      #crear la figura
    #p.line(source=source2, line_width=2, color="black")
    columns = [
        TableColumn(field="x", title="Fecha", formatter=DateFormatter(format="%Y-%m-%d"),width=700),
        TableColumn(field="y", title="Nivel"),
        TableColumn(field="x", title="Hora", formatter=DateFormatter(format="%H:%M:%S")),
        TableColumn(field="incremento", title="Incremento"),
    ]
    tabla = DataTable(source=source2, columns=columns, width=500, height=500,index_width=50, stylesheets=[css_personalizado])
    tablas.append(tabla)  # guardamos la tabla en la lista
    ###
    pico_1=p.scatter("x", "pico", source=source3, size=8, color="red", marker="triangle",legend_label="Picos (Pleamares)")

    # Para valles
    valle_1=p.scatter("x", "valle", source=source3, size=8, color="green", marker="inverted_triangle", legend_label="Valles (Bajamares)")
    ####################################################################################################
    #================================  FIGURA STUCKDATA  =======================================
    #linea
    linea3 = p.line('x_stuck','y_stuck',source= source_stuck, line_width=2, color="red", legend_label="Stuck data")
    # Estrellas en cada punto
    # Se importa Scatter en bokeh.models
    glyph = Scatter(x="x_stuck", y="y_stuck", size=12, marker="star", fill_color="gold")
    linea4 = p.add_glyph(source_stuck, glyph)

    # Crear y añadir la leyenda para las estrellas
    legend_item = LegendItem(label="Stuck points", renderers=[linea4])
    p.legend.items.append(legend_item)

    linea33.append(linea3)
    linea44.append(linea4)

    ####################################################################################################
    #================================  GUARDAR TODO EN LISTAS GLOBALES  =======================================
    pico_e.append(pico_1)
    valles_e.append(valle_1)

    lineas.append(linea1)#guardamos este conjunto de lineas
    labels_set = LabelSet(x='x', y='y', text='labels', source=source2, text_color="red", text_font_size="8pt")
    p.add_layout(labels_set)

    puntos.append(linea2)
    sources_E.append(source2)
    all_labels.append(labels_originales)  # guardamos originales
    # Hover para línea principal
    # Hover para todos los puntos
    # Hover solo para los puntos normales
    p.add_tools(HoverTool(
        renderers=[linea1],  # solo los puntos normales
        tooltips=[
            ("Fecha", "@x{%F}"),
            ("Nivel", "@y")
        ],
        formatters={'@x': 'datetime'}
    ))
  ####################################################################################################
  #================================  CASILLAS DE LAS LINEAS  =======================================
  #
  p.legend.location = "top_left"
  #ESTO ES PARA LA FIGURA
  p.xaxis.axis_label = "Fecha"
  p.yaxis.axis_label = "Nivel del mar (mm)"
  p.title.text_font_size = "20pt"
  p.toolbar.autohide = True
  p.toolbar.logo = None
  p.legend.location = "top_right"

  #ESTO ES PARA EL TITULO DE LA GRAFICA
  # Si quieres separarlos por coma
  titulo = ", ".join(nombres_archivos)

  # Si prefieres "y" cuando hay 2
  if len(nombres_archivos) == 2:
      titulo = " y ".join(nombres_archivos)
  elif len(nombres_archivos) > 2:
      titulo = ", ".join(nombres_archivos[:-1]) + " y " + nombres_archivos[-1]
  else:
      titulo = nombres_archivos[0]

  p.title.text = f"Gráfica del Archivo TOGA {titulo}"
  ####################################################################################################
  #================================  RANGO Y PROMEDIO =======================================

  p.legend.location = "top_left"

  # Controles
  chk_prom = Checkbox(label="Editar promedio", active=False)
  input_prom = TextInput(value=f"{prom_orig:.2f}", title="Promedio")

  chk_margin = Checkbox(label="Editar margen", active=False)
  input_margin = TextInput(value=f"{sigma_orig:.2f}", title="Margen (±)")

  # Div para mostrar resultados fuera de la gráfica
  resultados_div = Div(text=f"""
  <b>Promedio:</b> {prom_orig:.2f}<br>
  <b>Mín:</b> {min_orig:.2f}<br>
  <b>Máx:</b> {max_orig:.2f}
  """, width=200, height=80)


  # Callback = esto se ejecuta luego que el usuario realice alguna acción
  callback = CustomJS(args=dict(
      radio_sigma=radio_sigma,
      chk_prom=chk_prom, input_prom=input_prom,
      chk_margin=chk_margin, input_margin=input_margin,
      linea_prom=linea_prom, linea_min=linea_min, linea_max=linea_max,
      label_prom=label_prom, label_min=label_min, label_max=label_max,
      source_in_range=source_in_range,
      resultados_div=resultados_div,
      x=x_vals, y=y_vals,
      prom_orig=prom_orig, sigma_orig=sigma_orig,

  ), code="""
  // Solo actualizar si el checkbox está activo
      // Promedio
      let prom_val = chk_prom.active ? parseFloat(input_prom.value) : prom_orig;
      if (isNaN(prom_val)) prom_val = prom_orig;

      // Factor kr según selección del radio
      let kr = radio_sigma.active + 1;  // 0->1, 1->2, 2->3

      // Margen
      let margin_val = chk_margin.active ? parseFloat(input_margin.value) : sigma_orig;
      if (isNaN(margin_val)) margin_val = sigma_orig;

      // Calcular mínimo y máximo
      let min_val = prom_val - kr* margin_val;
      let max_val = prom_val + kr *margin_val;

      // Actualizar líneas
      linea_prom.data_source.data['y'] = [prom_val, prom_val];
      linea_min.data_source.data['y'] = [min_val, min_val];
      linea_max.data_source.data['y'] = [max_val, max_val];

      // Mostrar resultados en el Div
      resultados_div.text = "<b>Promedio:</b> " + prom_val.toFixed(2) +
                            "<br><b>Mín:</b> " + min_val.toFixed(2) +
                            "<br><b>Máx:</b> " + max_val.toFixed(2);

      // Mostrar en consola del navegador
      console.log(label_prom.text + " en y = " + prom_val.toFixed(2));
      console.log(label_min.text + " en y = " + min_val.toFixed(2));
      console.log(label_max.text + " en y = " + max_val.toFixed(2));

      // Actualizar etiquetas
      label_prom.y = prom_val;
      label_prom.text = "Promedio: " + prom_val.toFixed(2);
      label_min.y = min_val;
      label_min.text = "Mín: " + min_val.toFixed(2);
      label_max.y = max_val;
      label_max.text = "Máx: " + max_val.toFixed(2);


      // Filtrar puntos dentro del rango
      let xf = [];
      let yf = [];
      for (let i = 0; i < y.length; i++){
          if(y[i] >= min_val && y[i] <= max_val){
              xf.push(x[i]);
              yf.push(y[i]);
          }
      }
      source_in_range.data['x'] = xf;
      source_in_range.data['y'] = yf;
      source_in_range.change.emit();

      // Emitir cambios
      linea_prom.data_source.change.emit();
      linea_min.data_source.change.emit();
      linea_max.data_source.change.emit();
  """)

  # Vincular eventos
  for widget in [chk_prom, input_prom, chk_margin, input_margin]:
    widget.js_on_change("active" if isinstance(widget, Checkbox) else "value", callback)


  # Vincular el evento del RadioButton
  radio_sigma.js_on_change("active", callback)
  ####################################################################################################
  #================================  CASILLAS leyendas  =======================================
  #  checkbox_leyenda = CheckboxGroup(labels=["Mostrar Seno", "Mostrar Coseno", "Mostrar Seno+1"], active=[0,1,2])
  #Sirve para mostrar/ocultar líneas, puntos, picos, stuckData, etc., dinámicamente en el gráfico.
  #> de echo aqui puedes agregar mas callbacks para mostrar y ocultar leyendas
  checkbox_leyenda = CheckboxGroup(labels=["Puntos del nivel del mar","Dentro del rango","Promedio","Minimo","Maximo","Picos","valles","stuckData","stuckData estrella"], active=[0,1,2,3,4,5,6,7,8])

  callback_legend_vis = CustomJS(args=dict(legend=p.legend[0], checkbox=checkbox_leyenda), code="""
      for (let i = 0; i < legend.items.length; i++) {
          let show = checkbox.active.includes(i);
          legend.items[i].visible = show;
      }
  """)

  checkbox_leyenda.js_on_change('active', callback_legend_vis)

  select_leyenda = Select(title="Posición leyenda", value="top_right", options=[
      "top_left", "top_center", "top_right",
      "center_left", "center", "center_right",
      "bottom_left", "bottom_center", "bottom_right",
      "top", "left", "center", "right", "bottom"
  ])

  callback_legend_pos = CustomJS(args=dict(legend=p.legend[0]), code="""
      legend.location = cb_obj.value;
  """)

  select_leyenda.js_on_change('value', callback_legend_pos)
  ####################################################################################################
  #================================  CASILLAS DE LAS LINEAS  =======================================
  #STUCKDATA ESTRELLAS
  # Ocultar al inicio
  for l in linea33:
      l.visible = False
  for l in linea44:
      l.visible = False

  lineas33_44 = linea33 + linea44

  # Checkbox
  checkbox_StuckData = CheckboxGroup(labels=["Mostrar StuckData"])

  # Callback JS
  checkbox_StuckData.js_on_change("active",CustomJS(args=dict(lineas=lineas33_44), code="""
          const mostrar = this.active.includes(0); // solo una casilla
          for (let i = 0; i < lineas.length; i++) {
              lineas[i].visible = mostrar;
    }
  """))
  #ORIGINALES
  # Checkbox para controlar visibilidad
  checkbox_LiOriginal= CheckboxGroup(labels=nombres_archivos, active=list(range(len(nombres_archivos))))#creamos los chexkbox con forme a la cantidad de lineas

  # Callback JS para alternar visibilidad
  checkbox_LiOriginal.js_on_change("active", CustomJS(args=dict(lineas=lineas), code="""
      for (let i = 0; i < lineas.length; i++) {
          lineas[i].visible = this.active.includes(i);
      }
  """))
  #con nombre del archivo
  #checkbox_LiPuntos= CheckboxGroup(labels=nombres_archivos, active=list(range(len(nombres_archivos))))#creamos los chexkbox con forme a la cantidad de lineas
  checkbox_LiPuntos= CheckboxGroup(labels=[""] * len(nombres_archivos),active=list(range(len(nombres_archivos))))#creamos los chexkbox con forme a la cantidad de lineas
  #PUNTOS
  # Callback JS para alternar visibilidad
  checkbox_LiPuntos.js_on_change("active", CustomJS(args=dict(puntos=puntos), code="""
      for (let i = 0; i < puntos.length; i++) {
          puntos[i].visible = this.active.includes(i);
      }
  """))
  #TEXTO
  # JS callback para filtrar etiquetas
  checkbox_LiEtiqueta=CheckboxGroup(labels=[""] * len(nombres_archivos))
  callback_code = """
  for (let i = 0; i < sources_E.length; i++) {
      const source = sources_E[i];
      const original_labels = original_labels_list[i];
      source.data['labels'] = cb_obj.active.includes(i) ? original_labels : original_labels.map(() => "");
      source.change.emit();
  }
  """
  checkbox_LiEtiqueta.js_on_change("active", CustomJS(args=dict(
      sources_E=sources_E,
      original_labels_list=all_labels
  ), code=callback_code))

  #PICOS Y VALLES
  # Picos y Valles
  lineas_pv = [*pico_e, *valles_e]  # lista plana

  # Ocultar al inicio
  for l in lineas_pv:
      l.visible = False
  # Checkbox
  checkbox_pico_valle = CheckboxGroup(labels=["Mostrar Picos y Valles"])

  # Callback JS
  checkbox_pico_valle.js_on_change(
      "active",
      CustomJS(args=dict(lineas=lineas_pv), code="""
          const mostrar = this.active.includes(0); // solo una casilla
          for (let i = 0; i < lineas.length; i++) {
              lineas[i].visible = mostrar;
          }
      """)
  )

  ########################################################################################################
  #=====================  personlizar linea original de la grafica ======================================================

  # Supongamos que 'puntos' es un arreglo de GlyphRenderer (por ejemplo, resultado de p.circle(...))
  # 'colores' es un array de colores que corresponden a cada línea
  checkbox_colores = CheckboxGroup(labels=["azul", "Gris", "Naranja", "Morado"], active=[0])
  checkbox_1 = CheckboxGroup(labels=["Mostrar línea"], active=[0])
  text_input_1 = TextInput(value=p.title.text, title="Editar título:",width=900 )
  boton_1 = Button(label="Click aquí", button_type="success")

  # ====== 3. Callbacks JavaScript ======
  checkbox_colores.js_on_change("active", CustomJS(args=dict(lineas=lineas, colores=["blue","gray","orange","purple"]), code="""
    // Tomamos la primera casilla activa
    let color = cb_obj.active.length > 0 ? colores[cb_obj.active[0]] : "black";

    // Cambiamos el color de cada línea directamente
    for (let i = 0; i < lineas.length; i++){
        lineas[i].glyph.line_color = color;
        lineas[i].data_source.change.emit();  // fuerza que se dibuje con el nuevo color
    }
"""))
  text_input_1.js_on_change(
    "value",
    CustomJS(args=dict(figura=p), code="""
        figura.title.text = cb_obj.value;
    """)
  )
  #modifica tamaños de texto y separaciones para mejorar la legibilidad del gráfico.
  p.title.text_font_size = "20pt"
  p.xaxis.axis_label_text_font_size = "16pt"
  p.yaxis.axis_label_text_font_size = "16pt"
  # Separar labels del eje X
  p.xaxis[0].major_label_standoff = 15
  p.xaxis[0].axis_label_standoff = 25
  ####################################################################################################
  #================================ CREACIÓN DE ESCALAS, DEFINICION DE FORMTAO  =======================================

  # Formato inicial
  formatter = DatetimeTickFormatter(
      hours="%Y-%m-%a %d\n%H:%M:%S",
      days="%Y-%m-%a %d\n%H:%M:%S",
      months="%Y-%m-%a %d\n%H:%M:%S",
      years="%Y-%m-%a %d\n%H:%M:%S"
  )
  p.xaxis.formatter = formatter
  #p.xaxis.major_label_orientation = 1.57  # vertical
  p.xaxis.major_label_orientation = 0  # etiquetas horizontales

  # Ajustes iniciales
  p.xaxis[0].ticker.desired_num_ticks = 5
  p.yaxis[0].ticker.desired_num_ticks = 5

  # Sliders para divisiones
  slider_x = Slider(start=2, end=30, value=5, step=1, title="Divisiones en eje X")
  slider_y = Slider(start=2, end=30, value=5, step=1, title="Divisiones en eje Y")

  # Checkboxes para formato
  checkboxes_guia = CheckboxGroup(labels=["Año", "Mes", "Día", "Hora"], active=[0,1,2,3])

  # Selector para orientación
  orientation_select = Select(title="Orientación etiquetas X", value="Horizontal",
                              options=["Vertical", "Horizontal"])

  # JavaScript para sliders
  slider_x.js_on_change("value", CustomJS(args=dict(px=p.xaxis[0]), code="""
      px.ticker.desired_num_ticks = cb_obj.value;
  """))

  slider_y.js_on_change("value", CustomJS(args=dict(py=p.yaxis[0]), code="""
      py.ticker.desired_num_ticks = cb_obj.value;
  """))

  # JavaScript para checkboxes (formato dinámico con casos especiales)
  checkboxes_guia.js_on_change("active", CustomJS(args=dict(px=p.xaxis[0]), code="""
      const parts = {0: "%Y", 1: "%b", 2: "%a:%d", 3: "%H:%M:%S"};
      let active = cb_obj.active.sort();
      let fmt = active.map(i => parts[i]);

      let format_str = "";
      // Caso especial Día + Hora
      if (active.length === 2 && active.includes(2) && active.includes(3)) {
          format_str = "%a %d %H:%M:%S";
      }
      // Caso especial Año + Mes + Día
      else if (active.length === 3 && active.includes(0) && active.includes(1) && active.includes(2)) {
          format_str = "%Y %b %a:%d";
      }
      else {
          format_str = fmt.join(" ");
      }

      px.formatter.hours = format_str;
      px.formatter.days = format_str;
      px.formatter.months = format_str;
      px.formatter.years = format_str;
      px.change.emit();  // refrescar eje
  """))

  # JavaScript para orientación
  orientation_select.js_on_change("value", CustomJS(args=dict(px=p.xaxis[0]), code="""
      if (cb_obj.value === "Vertical") {
          px.major_label_orientation = 1.57;
      } else {
          px.major_label_orientation = 0;
      }
  """))
  ####################################################################################################
  #===============================  SELECTOR DE TABLA  =======================================
  tabla_layout =column(*tablas)
  i=0
  options = [f"Tabla {i+1}" for i in range(len(tablas))]
  select = Select(title="Seleccione tabla", value=options[0], options=options, width=200)

  # JS callback para cambiar visibilidad
  callback = CustomJS(args=dict(tabs=tablas, select=select), code="""
      const selected = select.value;
      for(let i=0; i<tabs.length; i++){
          if(selected === `Tabla ${i+1}`){
              tabs[i].visible = true;
          } else {
              tabs[i].visible = false;
          }
      }
  """)
  ####################################################################################################
  #================================  OBJETOS DIV CONTENIDO DE TEXTO  =======================================

  # Texto en blanco y negrita
  Espacio = Div(
      text="<span style='color:black; font-weight:bold;'>     </span>",
      width=300, height=60
  )

  Linea_original = Div(
      text="<span style='color:black; font-weight:bold;'>LINEA DE DATOS </span>",
      width=200, height=30
  )
  punto_tab = Div(
      text="<span style='color:black; font-weight:bold;'>PUNTOS </span>",
      width=100, height=30
  )
  etiqueta_tab = Div(
      text="<span style='color:black; font-weight:bold;'>ETIQUETA</span>",
      width=100, height=30
  )
  colores_linea = Div(
      text="<span style='color:black; font-weight:bold;'> COLORES DE LINEA  </span>",
      width=100, height=30
  )
  creador_escala = Div(
      text="<span style='color:black; font-weight:bold;'> CREAR MAS ESCALAS EN LOS EJES </span>",
      width=300, height=30
  )
  Leyendas_grafica = Div(
      text="<span style='color:black; font-weight:bold;'> LEYENDAS DE LAS GRAFICAS  </span>",
      width=300, height=30
  )
  cambiar_nombre = Div(
      text="<span style='color:black; font-weight:bold;'> CAMBIAR EL NOMBRE DE LA GRAFICA  </span>",
      width=300, height=40
  )
  promedio_general = Div(
      text="<span style='color:black; font-weight:bold;'> VER EL PROMEDIO DE LOS REGISTROS  </span>",
      width=300, height=40
  )
  ####################################################################################################
  #[]================================  ASIGNACIÓN DE NUEVAS FUNCIONES AGRUPADOS  =======================================
  #guia
  layout_guia = column(row(slider_x, slider_y),
    checkboxes_guia,
    orientation_select
  )
  #---
  #--- leyenda
  layout_leyenda = column(select_leyenda, checkbox_leyenda)


  select.js_on_change('value', callback)
  contenido_tabla = column(select, tabla_layout)
  ####################################################################################################
  #[]================================  SUBTAB PARA MAS FILTROS FUTUROS =======================================

  mensaje = Div(text="<h3 style='color:green;'>Aqui puedes incluir nuevos filtros</h3>") # estos es un objeto de tipo Div
  #------ SUGERENCIAS -----------
  #aqui puedes poner objetos de tipo column
  #COMO: Tab_14 = column(Espacio,checkbox_pico_valle,promedio_general,layout_rango)
  Tab_15 = TabPanel(child=mensaje, title="Parte 2")

  ####################################################################################################
  #=====================  Nuevos widgets  ======================================================
  # ====== 4. Contenedores ======
  # Columna de controles
  layout_rango = column(chk_todo ,column(chk_prom, input_prom),column(chk_margin, input_margin),column(radio_sigma,resultados_div))
  col_controles_1 = column (Leyendas_grafica, layout_leyenda)
  col_controles_2 = column(cambiar_nombre,text_input_1)
  col_controles_3 = row(col_controles_1,col_controles_2)
  col_controles = column(Espacio,creador_escala,layout_guia,col_controles_3, width=250)
  LinP = column(Espacio,Linea_original,checkbox_LiOriginal,colores_linea,checkbox_colores,width=250)
  LinP_2 = column(Espacio,punto_tab,checkbox_LiPuntos,width=80)
  LinP_3 = column(Espacio,etiqueta_tab,checkbox_LiEtiqueta,width=80)
  Tab_12 = row(LinP,LinP_2,LinP_3)
  Tab_14_1 = row(checkbox_pico_valle,checkbox_StuckData)
  Tab_14 = column(Espacio,Tab_14_1,promedio_general,layout_rango)

  Tab_16 = TabPanel(child=Tab_14, title="Parte 1")

  Tab_13 = Tabs(tabs=[Tab_16, Tab_15])
  #herramienta filtros
  #PRINCIPALES

  ####################################################################################################
  #=====================  CREACION DE TABS  ======================================================
  tab3_3 = TabPanel(child=Tab_13, title="Control de calidad")
  tab1 = TabPanel(child=p, title="GRAFICA")#Usar child=... porque TabPanel todavía acepta esa sintaxis
  tab2 = TabPanel(child=contenido_tabla, title="TABLA")
  tab3 = TabPanel(child=Tab_12, title="FILTROS")

  tab4 = TabPanel(child=col_controles, title="EDICION DEL LIENZO")
  ###

  for i, archivo in enumerate(archivos_cargados):
    print(f"Procesando archivo {i+1}/{len(archivos_cargados)}: {archivo}")

  #tab4 = TabPanel(child=checkbox, title="Cargar")

  #PRINCIPALES
  tabs = Tabs(tabs=[tab1])

  #HERRAMIENTAS DE FILTRADO AGRUPADOS
  tabs44 = Tabs(tabs=[tab3,tab2,tab4,tab3_3])

  general = row(tabs,tabs44)
  ####################################################################################################
  #===================== layout en HTML + JS embebido  ======================================================
  script, div = components(general)

  recursos = CDN.render()
  display(HTML(recursos + div + script))

  ####################################################################################################
  #===================== widgets para mostrar resutado de los scripts  ============================
  with Resultado_toga:
    clear_output()
    scripts = [
        "/content/MareograficoGoogleColab/ET1_ proyecto 1/GENERAL/SCRIPTS -- de practica/06_comprobar_datos_toga.py",
        "/content/MareograficoGoogleColab/ET1_ proyecto 1/GENERAL/SCRIPTS -- de practica/11_control_calidad_automatizado.py"
    ]

    widgets_script_1 = []

    for file_N in archivos_cargados:
        ejecucion_script = widgets.Output()  # ✅ nuevo Output por archivo
        with ejecucion_script:
            #Ejecuta varios scripts de Python sobre un archivo.
            for script in scripts:
                print("Archivo:", file_N)
                comando = ['python3', script, file_N]
                #corre un comando externo desde Python y se puede leer su salida
                resultado = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                print(resultado.stdout)
                if resultado.stderr:
                    print("⚠️ Error:", resultado.stderr)
                    print("")
            print("================================== FIN DEL ANALISIS ===============================")
            print("===================================================================================")
            print("")

        widgets_script_1.append(ejecucion_script)  # guardamos este Output individual

    # Creamos el VBox con todos los Output por archivo
    widgets_script = widgets.VBox(widgets_script_1)
    box45 = make_box("Ejecucion de Scripts", widgets_script)
    box45.layout.width = '1700px'
    box45.layout.height = '600px'
    box45.layout.justify_content = 'center'
    box45.layout.align_items = 'center'
    display(box45)

  ####################################################################################################
  #===================== widgets para mostrar la estructura de un archivo TOGA(datafreme) ==================
  with Archivo_toga:
    clear_output()
    file_N = []
    for file_N in archivos_cargados:
      print("archivos cargados",file_N)
      matriz_toga = pd.read_csv(file_N,sep=r'\s+',names =["StationID", "StationName", "Date", "D1", "D2", "D3","D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"], engine = 'python', skiprows = 1, na_values = "9999")
      #print("Matriz toga",matriz_toga.head())
      # version mejorada de google colab
      #display(matriz_toga.head()) # podemos ponerle valore al head(10) conforme a las filas que queremos ver
      # Crear widget interactivo
      # Ejemplo: botón para imprimir datos seleccionados
      output_TOGA = widgets.Output()
      with output_TOGA:
        pd.set_option("display.max_rows", None)   # Muestra todas las filas
        pd.set_option("display.max_columns", None)  # Muestra todas las columnas
        pd.set_option("display.width", None)     # No cortar líneas
        pd.set_option("display.max_colwidth", None)  # No truncar el contenido de celdas

        display(matriz_toga)  # ahora sí lo ves completo
      #contenedor de widgets
      #Configura tamaño y alineación
      widgets_toga=widgets.VBox([output_TOGA])
      box44 = make_box("ARCHIVO TOGA", widgets_toga)
      box44.layout.width ='1700px'
      box44.layout.height ='600px'
      box44.layout.justify_content = 'center'   # centra horizontal
      box44.layout.align_items = 'center'
      display(box44)

      #display(matriz_toga)  # Muestra las primeras filas
  # Crear la figura
###############################################################################
#==================== CARGADOR DE ARCHIVO =================================
# Inicializamos selector visible
contenedor_selector = widgets.Output()
Contenedor_salida= widgets.Output()
#tabs de
Archivo_toga = widgets.Output()
###############################################################################
#=================  ASIGNACION DE VARIABLES GLOBALES   ==========================
clear_output()
ruta_archivo=[]
#revisamos si no existe esta ruta
if not os.path.exists('/content/drive'):
  #usamos el modulo drive que conecta a la funcion mount, para asi montar nuestra ruta
  drive.mount('/content/drive')

#guardamos nuestra ruta base en ruta_
# ORIGINAL ------ ruta_drive = '/content/drive/MyDrive'
ruta_drive = '/content'

#mas adelante podemos modificaremos su contenido de forma dinamica
ruta_actual = [ruta_drive]
#=================  CARGAR ARCHIVOS  ==========================
# nuestra lista general de archivos
rutas_seleccionadas = []

#creamos nuestros contenedores para mostrar resultados
#con Output() creamos una area de salida interactiva, para mostrar errores y cualquier salida
#esto lo mostraremos mas adelante
salida_pc = widgets.Output()
salida_drive = widgets.Output()
salida_resultado = widgets.Output()
salida_seleccionado=widgets.Output()
output = widgets.Output()
#creamos dos boton que mostara un texto personalizado con estilo 'info'
#esto lo usaremos mas adelante
boton_pc = widgets.Button(description="Desde PC",button_style='info')
boton_drive = widgets.Button(description="Navegar en Drive",button_style='primary')
#================  CARGAR ARCHIVOS  ==========================
#creamos nuestra funcion para mostrar el contenido seleccionado desde drive
salida_seleccion_general = widgets.Output()

####################################################################################################
#[]===================== FUNCION QUE MUESTRA EL ARCHIVO SELECIONADO  ============================
def mostrar_contenido_drive():
  #usamos nuestro contenedor para mostrar el contenido que continua
  with salida_drive:
    #eliminamos todo lo anterior
    clear_output()
    with salida_seleccionado:
      #print("Has seleccionado",rutas_seleccionadas)
      clear_output()
      print("Has seleccionado")
      for iteracion in rutas_seleccionadas:
        print("-",iteracion)
      print("========================================")
    #obtenemos el primer elemento de la lista
    actual = ruta_actual[0]
    #listamos todas las carpetas(al igual que archivos) y las guardamos en archivos
    archivos = os.listdir(actual)

    #para cada elemento f  en la lista de archivos, si f es uan carpeta dentro de la ruta actual
    #agregar f a la lista de carpetas
    carpetas = sorted([f for f in archivos if os.path.isdir(os.path.join(actual,f))])

    #para cada elemento en la lista de archivos, si es un archivo agregalos a unalista
    #sorted nos ayuda a optener un alfabeticamente la lista interna y esa lista lo guardamos en ficheros
    ficheros = sorted([f for f in archivos if os.path.isfile(os.path.join(actual,f))])

    #cremos un widget de seleccion multiple
    #esto mostrara todos los archivos y podemos seleccionar varios
    selector = widgets.SelectMultiple(
        #unir las listas de carpetas y ficheros
        options = carpetas + ficheros,
        #agregamos una etiqueta de contenido
        description = 'Contenido:',
        #mostrar hasta 10 filas visibles a la vez
        rows = 10,
        #ajustamos el ancho del widget al 80%
        layout = widgets.Layout(width='80')
    )
    #creamos nuestros botones
    boton_entrar = widgets.Button(description="<Entrar a carpeta>", button_style='primary')
    boton_atras = widgets.Button(description="<--Ir atras",button_style='warning')
    boton_seleccionar = widgets.Button(description = ">> Seleccionar archivo <<", button_style='success')

    ####################################################################################################
    #[]===================== FUNCION QUE SIRVE PARA ABRIR CARPETAS  ============================
    #creamos nuestra funcion al hacer click en una carpeta
    def entrar_click(b):
      # leemos lo que el usuario ya ha seleccionado desde el widget en selector
      #convertimos esa seleccion en unalista y lo guardamos en la variable seleccion
      seleccion = list(selector.value)
      # si la cantidad de elementos seleccionados es = a 1, hacer lo siguiente
      if len(seleccion) ==1:
        #le asignamos a la variable nombre el primer elemento seleccionado
        nombre= seleccion[0]
        #recordemos que el ruta actual esta como  '/content/drive/MyDrive'
        #entonces le pasamos el nombre del archivo seleccionado
        #nueva ruta = tiene ahora la ruta del archivo seleccionado
        nueva_ruta = os.path.join(actual,nombre)

        #si la ruta guardada en 'nueva_ruta'es una carpeta
        #nueva_ruta = es igual al archivo seleccionado
        if os.path.isdir(nueva_ruta):
          # le pasamos la nueva ruta el primer elemento en posicion 0 de la lista de ruta actual
          ruta_actual[0] = nueva_ruta
          # y vamos de nuevo a mostrar el contenido de drive
          mostrar_contenido_drive()
        # y si no es asi pues
        else:
          #mostramos el widget de salida de resultado
          # donde limpiamos y imprimimos un mensaje
          with salida_resultado:
            clear_output()
            print("Solo puedes seleccionar una carpeta a la vez")
      #si no se ha seleccionado uno pues asemos lo siguiente
      else:
        #donde mostramos el siguiente mensaje de salida resultado
        with salida_resultado:
          clear_output()
          print("Solo puedes seleccionar una carpeta para entrar")

    # esto es para cuando hemos seleccionado nuestro boton de ir atras
    # a la funcion le pasamos el parametro b como representacion del boton presionado
    ####################################################################################################
    #===================== FUNCION QUE SIRVE RETROCEDER  ============================
    def atras_click(b):
      #preguntamos si ruta actual es diferente a ruta drive
      # si si estonces la condicion se cumple
      if ruta_actual[0] != ruta_drive:
        #cambiamos la ruta actual por {la carpeta anterior}
        ruta_actual[0] = os.path.dirname(ruta_actual[0])
        #de nuevo volvemos a iniciar
        mostrar_contenido_drive()
    ##################################|##################################################################
    #==================== FUNCION QUE SIRVE PARA SELECCIONAR ARCHIVOS MULTIPLES ============================
    # ahora con el boton de seleccionar el archivos
    def seleccionar_click(b):
      #leemos los elementos seleccioados  y con el widget de selector
      # ahora value devuelve los elementos seleccionado de forma tupla
      #('carpeta','archivo1.txt') pero con list lo convertimos en una lista
      seleccionados = list(selector.value)
      #inicializamos una variable
      agregados = 0

      #con la variable nombre recorremos la lista de elementos
      # ejemplo nombre se asigna ra el nombre de cada archivo acumulado en seleccionados
      #nombre = 'archivo.dat'
      with salida_seleccion_general:
        for nombre in seleccionados:
          #unimos la ruta actual con nombre que se esta iterando
          #actual es la ruta y en ruta completa se almacenara la ruta general
          ruta_completa = os.path.join(actual,nombre)
          #si existe el archivo en la ruta completa
          if os.path.isfile(ruta_completa):
            #si es asi entonces verificamos si ruta completa no esta en nuestra lista de rutas seleccionadas
            if ruta_completa not in rutas_seleccionadas:
              #si esto se comple
              #agregamos la ruta_completa asta el final de rutas seleccionadas
              rutas_seleccionadas.append(ruta_completa)
              #y incrementamos a agregados
              agregados =+1
            else:
              print("")
              print(f"==> Archivo duplicado (contenido igual) {os.path.basename(ruta_completa)} <==")
        #mostramos_rutas() es donde tenemos una funcion que muestra las ruta recien agregadas
        mostrar_rutas()
        #ademas verificamos si agregados fue incrementado
        # es para saber si hubo archivos añadidos
        if agregados == 0:
          print("Ningun archivo nuevo fue agregado")
    #cuando presionemos el boton vamos a entrar a ejecutar sus funciones
    boton_entrar.on_click(entrar_click)
    boton_atras.on_click(atras_click)
    boton_seleccionar.on_click(seleccionar_click)

    #mostramos una caja vertical
    display(widgets.VBox([
        #usamos un texto html que dice ruta actual en negrita
        widgets.HTML(f"<b>Ruta actual:</b>{actual}"),
        #el selector de opcion
        selector,
        #una fila horizontal con los botones atras, entrar y seleccionar
        widgets.HBox([boton_atras,boton_entrar,boton_seleccionar])
    ]))
# un conjunto vacio de coleciones de elementos unico y no ordenados
hashes_archivos = set()
####################################################################################################
#===================== FUNCION QUE SIRVE PARA LEER EL ARCHIVO EN BLOQUES  ======================
def obtener_hash(archivo_path):
  hash_md5 = hashlib.md5()
  with open(archivo_path, "rb") as f: # Abrir archivo en modo binario
    for chunk in iter(lambda:f.read(4096),b""):#  Leer en bloques de 4096 bytes
      hash_md5.update(chunk)
  return hash_md5.hexdigest()
####################################################################################################
#===================== FUNCION QUE SIRVE PARA LEER EL ARCHIVO EN BLOQUES  ======================
def seleccionar_desde_pc(b=None):
  with salida_pc:
    clear_output()
    print("Sube un archivo desde tu PC:")
    #print("Has seleccionado",rutas_seleccionadas)
    #print("Rutas Guardadas",ruta_archivo)
    upload = files.upload()

    for nombre in upload.keys():
      ruta_local = os.path.abspath(nombre)
      hash_archivo = obtener_hash(ruta_local)
      if hash_archivo not in hashes_archivos:
        rutas_seleccionadas.append(ruta_local)
        hashes_archivos.add(hash_archivo)
        print("")
        print(f"==> Archivo añadido:{os.path.basename(ruta_local)} <==")
      else:
        print("")
        print(f"==> Archivo duplicado (contenido igual){os.path.basename(ruta_local)} <==")
        print("")
    mostrar_rutas()

####################################################################################################
#===================== MUESTRA ARCHIVS SELECCIONADOS ======================
def mostrar_rutas():
    with salida_resultado:
        clear_output()
        if rutas_seleccionadas:
            print("")
            print("Archivos seleccionados")
            print("")
            for i, ruta in enumerate(rutas_seleccionadas, 1):
                print(f"{i}. {ruta}")
                ruta_archivo.append(ruta)
        else:
            print("No se ha seleccionado ningún archivo aún")
    #print("Has seleccionado",rutas_seleccionadas)
    with Contenedor_salida:
      clear_output()
      actualizar_selector()
####################################################################################################
#===================== MUESTRA RESULTADO FINAL ======================
def activar_pc(b):
  salida_drive.clear_output()
  salida_resultado.clear_output()
  ruta_actual[0]=ruta_drive
  seleccionar_desde_pc(None)

def activar_drive(b):
  salida_pc.clear_output()
  salida_resultado.clear_output()
  ruta_actual[0]=ruta_drive
  mostrar_contenido_drive()

#==================== CARGADOR DE ARCHIVO =================================
# Archivos "pre-cargados"
#aqui como es una lista usamos un for
nombre=[]

archivos_disponibles = rutas_seleccionadas

from os.path import basename

# Crear opciones como lista de tuplas (nombre_visible, ruta_real)
opciones = [(os.path.basename(r), r) for r in archivos_disponibles]

###############################################################################
#==================== CARGADOR DE ARCHIVO =================================

# Checkbox para activar función (que cambia tipo de selección)
checkbox_activar_funcion = widgets.Checkbox(
    value=True,
    description="Selector multiple"
)
#==================== CARGADOR DE ARCHIVO =================================
# Selector de archivos (se cambiará dinámicamente entre Select y SelectMultiple)
with Contenedor_salida:
    clear_output()
    selector_archivos_multiple = widgets.SelectMultiple(
        options=opciones,
        value=[],              # elementos seleccionados por defecto
        description="Archivos:",
        rows=6
    )
selector_archivos_multiple.layout = widgets.Layout(width='400px', height='200px')
selector_archivos_multiple.style = {'description_width': 'initial'}  # evita que la etiqueta se corte


# Inicialmente usamos selector simple
selector_archivos_actual = selector_archivos_multiple
# Botón procesar
boton_procesar = widgets.Button(
    description="Procesar",
    button_style='info'  # opciones: 'primary', 'success', 'info', 'warning', 'danger', ''
)

#==================== CARGADOR DE ARCHIVO =================================
# Output detalles en pestaña 2
output_detalle = widgets.Output()
Resultado_toga = widgets.Output()
#==================== Grafica Dividida =================================
Graficar = widgets.Output()
Captura_tabla_general = widgets.Output()
#==================== CARGADOR DE ARCHIVO =================================

###############################################################################
#==================== FUNCION DEL SELECTOR DE ARCHIVOS =====================
def actualizar_selector():
    global selector_archivos_actual
    if checkbox_activar_funcion.value:
        # Función activada: usamos SelectMultiple para permitir seleccionar varios
        selector_archivos_actual = selector_archivos_multiple
        #nombre = [os.path.basename(ruta) for ruta in rutas_seleccionadas]
        #selector_archivos_actual.options = nombre
        selector_archivos_actual.options = archivos_disponibles

def cambiar_selector(change=None):
    # Cuando cambia el checkbox, actualizamos el selector visible y texto
    actualizar_selector()
    # Actualizamos texto
    # Reemplazamos en el layout
    # Quitamos widget viejo y ponemos nuevo
    with contenedor_selector:
        clear_output()
        display(selector_archivos_actual)

checkbox_activar_funcion.observe(cambiar_selector, names='value')


with contenedor_selector:
    display(selector_archivos_actual)
###############################################################################
#====================VARIABLES DE LA GRAFICA =================================
dfs = []
archivos_cargados = []
#################################################################
#==================== Ventana =================================

casilla_1 = widgets.Checkbox(
    value=False,
    description='Analizar y Graficar',
    disabled=False,
    indent=True
)

casilla_2 = widgets.Checkbox(
    value=False,
    description='Resumen general',
    disabled=False,
    indent=True
)

salida_1 = widgets.Output() # salidad de los mesajes
salida_2 = widgets.Output() # salidad de los mesajes
salida_panel = widgets.Output() # salidad de los mesajes
###############################################################################
#==================== ES PARA EL BOTON QUE PROCESA LOS ARCHIVOS  ==========
archivos_procesado_1=[]
increm = 0
autonomo = 0
def procesar_click(b):
    global archivos_procesados,archivos_procesado_2,dfs, archivos_cargados, tabla_local, tablas,linea1,increm,autonomo # <- importante
    # Tomamos seleccionados según el modo
    seleccionados = list(selector_archivos_actual.value)

    archivos_procesados = seleccionados
    archivos_procesado_2 = archivos_procesados


    with output_detalle:
        clear_output()
        print("")
        print("Archivos procesados:")
        increm += 1
        for f in archivos_procesados:
            print(f"- {f}")

            #print(increm)
            # Tomar solo el primer archivo válido
            #es para refrescar nuestro programa
        ##########################################################################################################################
        #==================== ESTO SE INICIALIZA SOLO LA PRIMERA VEZ QUE SE SELECCIONA UN ARCHIVO EN EL PROGRAMA ==========
        if increm == 1:
          with salida_1:  # enviamos salida a salida_1
            archivos_procesado_2 = archivos_procesados[:1]
            if archivos_procesado_2:  # verificamos que exista
                if casilla_1.value:
                    archivos_procesado_2 = archivos_procesados
                    # ya está prendida → usamos su lógica normal
                    evento_casilla_1({"new": casilla_1.value})  # asegura estado prendido
                    #casilla_1.value = False
                    autonomo = 1

                else:
                    # estaba apagada → la prendemos y disparamos la lógica
                    casilla_1.value = True
                    #casilla_1.value = False
                    autonomo = 1
                archivos_procesado_2.clear()
            else:
                print("⚠️ No se seleccionó ningún archivo válido")

        ##########################################################################################################################
        #==================== ESTO ES UN SEGUNDO PASO PARA RECARGAR LA CASILLAS DE SIERTOS MODOS  ==========
        #tambien nos sirve para mandar advertencias al seleccionar algun archivo fuera del rango
        rango = len(archivos_procesados)
        if rango == 0:
          print("NO has selecionado ningun archivo")
        if rango >= 3:
          print("")
          print("SELECCIONO",rango)
          print("EFECTOS EN LOS MODOS DE ANALISIS")
          print("Para ---- Resumen general ---- normal")
          print("Para ---- Analizar y graficar ----Muy Lento")
          print("⚠️Tenga cuidado al usar esta funcion puede que se relentice la interaccion ⚠️")
        # 🔍 Verificamos estados de casillas
        with salida_panel:
          clear_output()
          if autonomo == 1:
            autonomo=0
          else:
              if casilla_1.value:
                  #print("✔ Analisis automatico activado")
                  #print("⚠️Tenga cuidado al seleccionar mas de 3 archivos⚠️")
                  #aquí puedes llamar evento_casilla_1(None) si quieres ejecutar su lógica
                  with salida_1:  # <- enviamos la salida a salida_1
                      evento_casilla_1({"new": casilla_1.value})  # simulamos el "change"
              #else:
                  #print("❌ Casilla 1 está desactivada")

              #if casilla_2.value:
                  #print("✔ Casilla 2 está activa")
                  #with salida_2:  # <- enviamos la salida a salida_1
                  #    evento_casilla_2({"new": casilla_2.value})  # simulamos el "change"
              #else:
                  #print("❌ Casilla 2 está desactivada")
########################################################################################
#==================== DEFINIMOS TODOS LOS WIDGETS =================================
#definimos auque esten globales por que nos aparece un error
boton_procesar.on_click(procesar_click)
archivos_cargados = []
tabla_local = []
tablas = []
linea1=[]
#=================== CARGADOR DE ARCHIVO =================================
# Layout pestaña 1
tab1 = widgets.VBox([
    widgets.Label(value="Aqui veras el el analisis de los archivos"),#contenido texto
    contenedor_selector,
    boton_procesar,
])
# Layout pestaña 2
tab2_resultado = widgets.VBox([
    output_detalle
])

#=================  CARGAR ARCHIVOS  ==========================
boton_pc.on_click(activar_pc)
boton_drive.on_click(activar_drive)
salida_resultado.layout = widgets.Layout(margin='0 10px 0 0')  # margen derecho
salida_drive.layout = widgets.Layout(margin='0 10px 0 10px')   # márgenes laterales
salida_pc.layout = widgets.Layout(margin='0 0 0 10px')          # margen izquierdo
salida_seleccionado.layout = widgets.Layout(margin='0 0 0 10px')          # margen izquierdo
#=================  Ventana  ==========================
contenedor = widgets.Label("No hay datos")
##############################################################################################################################
#==================== DEFINIMOS EL METODO DE LA GRAFICA Y PROCESAMOS NUESTRO ARCHIVO TOGA=================================
def evento_casilla_1(change):
  with salida_1:
    clear_output()
    salida_1.clear_output()
    if change['new']:
      print("Procesando...")
      #[A2]==================== Grafica Dividida =================================
      #LOGICA DE GRAFICA

      with Graficar:
        clear_output()
        def analisador_archivos():
          if not archivos_procesado_2:
            print("No se cargaron archivos.")
            return


          # Ya NO hagas dfs = [] aquí, solo modifica la global si quieres reiniciar
          # Si quieres reiniciar los datos, hazlo aquí:
          dfs.clear()
          archivos_cargados.clear()
          tabla_local.clear()
          tablas.clear()
          linea1.clear()

          for archivo in archivos_procesado_2:
            #importa un archivo estructurado con espacios en columnas, asigna nombres y maneja valores nulos.
            matriz_toga = pd.read_csv(archivo, sep=r'\s+', names=["StationID", "StationName", "Date",
                                "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"],
                                engine='python', skiprows=1, na_values="9999")

            lista_fecha = []
            lista_datos = []

            for filas_dat in matriz_toga.index:
                fecha = str(matriz_toga["Date"][filas_dat])
                if fecha[8:9] == "1":
                    fecha_inicial = datetime.datetime(int(fecha[:4]), int(fecha[4:6]), int(fecha[6:8]))
                else:
                    fecha_inicial = datetime.datetime(int(fecha[:4]), int(fecha[4:6]), int(fecha[6:8]), 12)

                for dat in ["D1", "D2", "D3", "D4",  "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"]:
                    lista_fecha.append(fecha_inicial)
                    lista_datos.append(matriz_toga[dat][filas_dat])
                    fecha_inicial += datetime.timedelta(hours=1)
            ##############################################################################################################################
            #==================== METODO PARA MOSTRAR LINEA RECTA en la grafica original(stuckData)=================================
            stucktotal = 0
            stuckcount = 0                  # Contador de valores consecutivos iguales
            stucklimit = 4                  # Cantidad de valores consecutivos  a partir de la cual la serie se considera atorada
            stuckvalue = lista_datos[0]
            stuckindex = 0

            lista_stuck_datos = list()
            lista_stuck_fechas = list()

            # Recorrer todo el arreglo de datos
            for ind in range(len(lista_datos)):

                # Comenzar a partir del segundo dato
                if ind > 0:

                    # Verificar si el dato que se está revisando actualmente no es igual al stuckvalue
                    if stuckvalue == lista_datos[ind] and ind-1 == stuckindex:
                        stuckcount = stuckcount + 1
                    else:
                        stuckcount = 0

                    if stuckcount >= stucklimit-1:

                        # Si se trata de los primeros tres valores, etiquetar los dos valores previos
                        if stuckcount == stucklimit-1:
                            for k in range(ind-(stucklimit-1), ind):
                                #print("---Se han encontrado y etiquetado valores iguales consecutivos (stuck values). Valor: "+str(lista_datos[k])+" Posicion: "+str(k))
                                lista_stuck_datos.append(lista_datos[k])
                                lista_stuck_fechas.append(lista_fecha[k])
                            stucktotal = stucktotal + 1


                        #print("---Se han encontrado y etiquetado valores iguales consecutivos (stuck values). Valor: "+str(lista_datos[ind])+" Posicion: "+str(ind))
                        lista_stuck_datos.append(lista_datos[ind])
                        lista_stuck_fechas.append(lista_fecha[ind])

                    stuckvalue = lista_datos[ind]
                    stuckindex = ind


            df = pd.DataFrame({"Date": lista_fecha, "Registro": lista_datos})
            #----- explicacion ---------
            # agregar las columnas “stuck” al DataFrame original sin preocuparte de que las listas tengan distinta longitud
            df['Date_stuck'] = pd.Series(lista_stuck_fechas)
            df['Registro_stuck'] = pd.Series(lista_stuck_datos)

            dfs.append(df)
            archivos_cargados.append(archivo)

        analisador_archivos()

        actualizar_vista()
    else:
      print("Desactivado")
############################################################################################################
#==================== AQUI CONSTRUIMOS EL EVENTO DEL RESUMEN GENERAL =================================
#aqui nos encargamos de llamar nuestro modulo llamado sincronizado.py
archivo_lista = []
def evento_casilla_2(change):
  with salida_2:
    salida_2.clear_output()
    if change['new']:
      print("Realizando resumen...")


      #Archivos seleccionados
      #aqui va tambien la logica que llama la funcion
      # eso si estara en forna de windgets para que el resultado se vea en un tab y no aqui
      #with llamada_1:
        #print('Hola Mundo')
      archivos_data = {}  # clave = nombre, valor = contenido

      for path in archivos_procesados:
          nombre = os.path.basename(path)  # solo el nombre del archivo
          with open(path, 'r') as f:
              contenido = f.read()
          archivos_data[nombre] = contenido

      #print(archivos_data.keys())  # nombres de archivos
      #print(archivos_data)  # contenido del archivo
      contenedor = super_resumen(archivos_data)

      with Contenedor_salida:
        clear_output()
        display(contenedor)
    else:
      print("Desactivado")
###############################################################################
#==================== CONTRUCCION DE WIDGETS =================================
casilla_1.observe(evento_casilla_1, names='value')
casilla_2.observe(evento_casilla_2, names='value')

division_casilla = widgets.HBox([casilla_1,salida_1])
division_casilla_1 = widgets.HBox([casilla_2,salida_2])

#================  CARGAR ARCHIVOS  ==========================

# Agruparlos en una fila (HBox) y centrar
hbox_botones = widgets.HBox(
      [boton_pc, boton_drive],
      layout=widgets.Layout(justify_content='center')
  )
# Diseño a los salidas
hbox_botones2 = widgets.VBox(
      [salida_drive,salida_pc,salida_seleccion_general],
      layout=widgets.Layout(justify_content='center')
)
###############################################################################
#================  Cuadros  ==========================
# Ejemplo de contenido: widget con texto largo para activar scroll
resultado_general = widgets.VBox(
    [salida_seleccionado,salida_resultado],
    layout=widgets.Layout(justify_content='center')

)
#=================  Cuadros  ==========================
# Otro contenido: lista de botones (para mostrar ejemplo variado)
#long_text2 = widgets.HTML(
#    value="<br>".join([f"Línea {i}" for i in range(50)])
#)
#=================  Cuadros  ==========================
# Crear cajas con contenido variado
box1 = make_box("Cargar Archivo",hbox_botones)
box1.layout.width ='2000px'
box1.layout.height ='100px'
box1.layout.justify_content = 'center'   # centra horizontal
box1.layout.align_items = 'center'
box2 = make_box("Resultado",resultado_general)
box2.layout.width ='1000px'
box2.layout.height ='500px'
box3 = make_box("Archivos cargados",hbox_botones2)
box3.layout.width ='1000px'
box3.layout.height ='500px'

#=================  Cuadros  ==========================
# Organizar cajas en grid visual con HBox y VBox
fila = widgets.HBox([box1])
fila1 = widgets.HBox([box3,box2])
dashboard = widgets.VBox([fila,fila1])
###############################################################################
# =================  Ventana  ==========================
#VENTANA 2---- contenido de ventana 2
# Contenido para pestañas internas de la pestaña anidada
seleccion_final = widgets.HBox([
    tab1,salida_panel
],layout=widgets.Layout(
        width="100%",
        height="300px",              # altura del área
        border="3px solid #2D6FA4",     # solo para que veas el contenedor
        padding="1px"
    ))
seleccion_final_3 = widgets.VBox([
    output_detalle
],layout=widgets.Layout(
        width="100%",
        height="300px",              # altura del área
        border="3px solid #2D6FA4",     # solo para que veas el contenedor
        padding="1px"
    ))
seleccion_final_2 = widgets.VBox([
    widgets.Label(value="Modo analisis"),#contenido texto
    division_casilla,
    division_casilla_1
],layout=widgets.Layout(
        width="100%",
        height="300px",              # altura del área
        border="3px solid #2D6FA4",     # solo para que veas el contenedor
        padding="1px"
    ))
inner_tab1 = widgets.HBox([

    seleccion_final_2,seleccion_final_3,
    seleccion_final
    ])
#Texto para el panel
title = widgets.HTML(
    value="""
    <h1 style='text-align:center;'>Panel para seleccionar archivos</h1>
    <h4 style='text-align:center;'>[ Analizar y graficar ] = sirve para graficar y aplicar filtros, pero tambien para analizar con mayor detalle. Puedes graficar mas archivos juntos maximo 3 </h4>
    <h4 style='text-align:center;'>[ Resumen general ]  = sirve para analizar muchos archivos de forma general, puedes seleccionar mas de 40 archivos ala vez</h4>
    """
)
contenido = widgets.VBox([title,inner_tab1])
accordion = widgets.Accordion(children=[contenido])
accordion.set_title(0,"<<  Mostrar / Ocultar  >>")
tab1_content = widgets.VBox([
    dashboard,
])

#VENTANA 1 --- sus demas comparñeros
#################################################################################################
#=================  Ventanas principales y generales con ipywidgets ==========================
tab2_content = widgets.VBox([
    #Ventana 2
    #aqui va la grafica
    #inner_tabs,  # Aquí va la ventana anidada dentro de la pestaña 1
    accordion,
    Graficar
    ])
tab3_content = widgets.VBox([
    widgets.Label(value="Aqui veras el analisis de los archivos TOGA"),#contenido texto
    Resultado_toga
])
tab4_content = widgets.VBox([
    widgets.Label(value="Estructura del archivo TOGA"),#contenido texto
    Archivo_toga
])
tab5_content = widgets.VBox([
    widgets.Label(value="Aqui puedes visualizar muchos archivos TOGA"),#contenido texto
    salida_2,Contenedor_salida
])
tab6_content = widgets.VBox([
    widgets.Label(value="Aqui veras el el analisis de los archivos"),#contenido texto

])
# Crear el widget de pestañas principal
#=================  Ventana  ==========================
tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content,tab4_content,tab5_content])# [la llamada]
tabs.set_title(0, 'INICIO') # ===El nombre====
tabs.set_title(1, 'GRAFICA')
tabs.set_title(2, 'Analisis del archivo')
tabs.set_title(3, 'Archivo Toga') # ===El nombre====
tabs.set_title(4, 'Resumen General')
#####################################################################################################
# =================  bloque de HTML y CSS embebido__ INSERCIÓN DE IMAGEN  ==========================
html_con_estilo = f"""
<div style="width:100%; height:100%; border:0px solid #ccc; padding:0; margin:0;">
    <img src="https://i.postimg.cc/4dXdpScF/Screenshot-2.png"
         alt="Ejemplo"
         style="width:100%; height:100%; object-fit:cover; display:block; margin:auto;">
</div>
"""

#Mostramos contenido HTML
display(HTML(html_con_estilo))


# Mostrar la ventana principal
display(tabs)
#####################################################################################################
# =================  bloque de HTML y CSS embebido__Fondo estilizado  ==========================
vista = f"""
  <div style="min-height:300px; border: 1px solid #ccc; padding:10px;background-color:#2D6FA4">
  </div>
  """
display(HTML(vista))


Mounted at /content/drive


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#ESTO NOS SIRVE PARA BORRAR LOS ARCHIVOS .DAT EN CONTENT(donde se cargan y se almacenan carpetas y archivos)
#EJECUTAR  ESTA CELDA solo cuando se quiere restablecer y mantener nuevos archivos
#pero si quieres restablecer todo es mejor detener las sesiones en "ADMINISTRADOR DE SESIONES" pero ojo esta acción te hara iniciar ejecutando todo de nuevo
#asi que para ahorrate eso, mejor restablece los arhivos, solo aplica con archivos:
# /content/h394a74.dat
# /content/h394a72.dat
# /content/h394a67.dat
# /content/h394a83.dat
# /content/h316a55.dat
# /content/h316a52.dat
# /content/h394a68.dat

import os
import glob

# Buscar todos los .dat en /content
archivos = glob.glob("/content/*.dat")

for archivo in archivos:
    os.remove(archivo)
    print(f"Borrado: {archivo}")
